In [ ]:
import pyodbc
import random
import time

# Establish a connection to the SQL Server database
connection = pyodbc.connect('DRIVER={SQL Server};'
                            'SERVER=DESKTOP-DDT3O6B\\SQLEXPRESS;'
                            'DATABASE=newdb;'
                            'Trusted_Connection=yes;')

# Create a cursor object
cursor = connection.cursor()

# Define the SQL query to create the qualification table if it doesn't exist
create_table_query = '''
IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='qualification')
BEGIN
    CREATE TABLE qualification (
        id INT PRIMARY KEY IDENTITY,
        driver_name VARCHAR(255) NOT NULL,
        lap_time VARCHAR(12) NOT NULL,
        sector1 FLOAT NOT NULL,
        sector2 FLOAT NOT NULL,
        sector3 FLOAT NOT NULL,
        lap INT NOT NULL,
        created_at DATETIME DEFAULT GETDATE()
    );
END
'''

# Execute the create table query
cursor.execute(create_table_query)
connection.commit()

# Calculate the total number of laps per driver
total_laps_per_driver = 100
driver_names = ["Speedy Gonzalez", "Road Runner", "Granny Gear", "Turbo Tim"]

# Generate and insert lap times for each driver in a round-robin fashion
for lap_number in range(1, total_laps_per_driver + 1):
    for driver_name in driver_names:
        # Generate random sector times with three decimal places
        sector1 = round(random.uniform(25, 29), 3)  # seconds for sector 1
        sector2 = round(random.uniform(32, 35), 3)  # seconds for sector 2
        sector3 = round(random.uniform(28, 31), 3)  # seconds for sector 3

        # Calculate total lap time
        lap_time_seconds = sector1 + sector2 + sector3
        lap_time_minutes = int(lap_time_seconds // 60)
        lap_time_seconds %= 60
        lap_time_decimals = int((lap_time_seconds % 1) * 1000)
        lap_time = "{:01d}:{:02d}:{:03d}".format(lap_time_minutes, int(lap_time_seconds), lap_time_decimals)

        # Insert data into SQL Server database
        insert_query = """
            INSERT INTO qualification (driver_name, lap_time, sector1, sector2, sector3, lap) 
            VALUES (?, ?, ?, ?, ?, ?)
        """
        data = (driver_name, lap_time, sector1, sector2, sector3, lap_number)
        cursor.execute(insert_query, data)
        connection.commit()

    # Delay for 3 seconds before the next round of laps
    time.sleep(3)

# Close the cursor and connection
cursor.close()
connection.close()